In [42]:
import subprocess
import cv2
import matplotlib.pyplot as plt
import subprocess
import time
import os
import glob

In [43]:
def get_image_paths(image_folder):
    # Get the list of all image files in the specified folder
    image_paths = glob.glob(os.path.join(image_folder, '*'))
    return image_paths

In [44]:
def run_ilastik(input_path, ilastik_script_path, project_path, result_base_path, export_source="Simple Segmentation", output_format="png"):
    """
    This function runs the Ilastik software in headless mode with the specified parameters.

    Parameters:
    input_path (str): The path to the image file or folder to be processed.
    ilastik_script_path (str): The path to the Ilastik script.
    project_path (str): The path to the Ilastik project file.
    result_base_path (str): The base path where the result will be saved.
    export_source (str, optional): The type of data to export. Default is "Simple Segmentation". Must be one of ["Probabilities", "Simple Segmentation", "Uncertainty", "Features", "Labels"].
    output_format (str, optional): The format of the output file. Default is "png". Must be one of ["bmp", "gif", "hdr", "jpeg", "jpg", "pbm", "pgm", "png", "pnm", "ppm", "ras", "tif", "tiff", "xv", "bmp sequence", "gif sequence", "hdr sequence", "jpeg sequence", "jpg sequence", "pbm sequence", "pgm sequence", "png sequence", "pnm sequence", "ppm sequence", "ras sequence", "tif sequence", "tiff sequence", "xv sequence", "multipage tiff", "multipage tiff sequence", "hdf5", "compressed hdf5", "numpy, dvid"].

    Returns:
    None

    Raises:
    ValueError: If export_source or output_format is not in the allowed list.
    subprocess.CalledProcessError: If there is an error during the execution of the Ilastik command.
    """
    
    ALLOWED_SOURCES = ["Probabilities", "Simple Segmentation", "Uncertainty", "Features", "Labels"]
    ALLOWED_FORMATS = ["bmp", "gif", "hdr", "jpeg", "jpg", "pbm", "pgm", "png", "pnm", "ppm", "ras",
                    "tif", "tiff", "xv", "bmp sequence", "gif sequence", "hdr sequence", "jpeg sequence",
                    "jpg sequence", "pbm sequence", "pgm sequence", "png sequence", "pnm sequence",
                    "ppm sequence", "ras sequence", "tif sequence", "tiff sequence", "xv sequence",
                    "multipage tiff", "multipage tiff sequence", "hdf5", "compressed hdf5", "numpy, dvid"]
    
    if export_source not in ALLOWED_SOURCES:
        raise ValueError(f"Invalid export_source. Allowed values are {ALLOWED_SOURCES}")
    
    if output_format not in ALLOWED_FORMATS:
        raise ValueError(f"Invalid output_format. Allowed values are {ALLOWED_FORMATS}")
    
    # Check if result_base_path exists, if not, create it
    if not os.path.exists(result_base_path):
        os.makedirs(result_base_path)

# Check if input_path is a directory or a file
    if os.path.isdir(input_path):
        image_arg = get_image_paths(input_path)
        # Arguments to execute Ilastik in headless mode
        ilastik_args = [
            ilastik_script_path,
            "--headless",
            "--project=" + project_path,
            "--export_source=" + export_source,
            "--output_format=" + output_format,
            "--output_filename_format=" + result_base_path + "{nickname}_" + export_source.replace(" ", "_"),
            *image_arg
        ]
    else:
        image_arg = input_path
        # Arguments to execute Ilastik in headless mode
        ilastik_args = [
            ilastik_script_path,
            "--headless",
            "--project=" + project_path,
            "--export_source=" + export_source,
            "--output_format=" + output_format,
            "--output_filename_format=" + result_base_path + "{nickname}_" + export_source.replace(" ", "_"),
            image_arg
        ]

    # Execute the Ilastik command in headless mode with the specified arguments
    try:
        subprocess.run(ilastik_args, check=True)
        print(f"Conversion of {input_path} completed successfully")
    except subprocess.CalledProcessError as e:
        print("Error during conversion : ", e)
        raise RuntimeError("Error during Ilastik execution. See console output for details.")

In [22]:
def display_images(image_path, result_base_path, export_source, output_format):
    """
    This function reads an original image and its corresponding result image, concatenates them horizontally, and displays the concatenated image.

    Parameters:
    image_path (str): The path to the original image file.
    result_base_path (str): The base path where the result image is saved.
    export_source (str): The type of data that was exported to create the result image.
    output_format (str): The format of the result image file.

    Returns:
    None

    Note:
    The function assumes that the result image file is named by replacing the ".jpg" extension of the original image file name with "_{export_source}.{output_format}", where {export_source} is the export_source parameter with spaces replaced by underscores.

    The function uses OpenCV to read the images and Matplotlib to display the concatenated image. The images are concatenated horizontally using cv2.hconcat. The concatenated image is displayed in RGB color space, so the function converts the color space from BGR to RGB using cv2.cvtColor.
    """
    
    input_filename = os.path.basename(image_path)

    result_path = os.path.join(result_base_path, input_filename.replace(".jpg", "_" + export_source.replace(" ", "_") + "." + output_format))

    img = cv2.imread(image_path)
    result = cv2.imread(result_path)

    concatenated_img = cv2.hconcat([img, result])

    plt.figure(figsize=(15, 15))
    plt.imshow(cv2.cvtColor(concatenated_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

In [40]:
input_path = "/Users/titouanlegourrierec/Desktop/selection_feuilles/TITOUAN"
ilastik_script_path = "/Applications/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/run_ilastik.sh"
project_path = "/Users/titouanlegourrierec/Desktop/labelisation3.ilp"
result_base_path = "/Users/titouanlegourrierec/Desktop/ilastik_result/"
export_source = "Simple Segmentation"
output_format = "png"

start_time = time.time()
run_ilastik(input_path = input_path,
            ilastik_script_path = ilastik_script_path,
            project_path = project_path,
            result_base_path = result_base_path,
            export_source = export_source,
            output_format = output_format)
end_time = time.time()
print(f"Execution time: {(end_time - start_time):.3f} seconds")

'''
display_images(image_path = image_path,
               result_base_path = result_base_path,
               export_source = export_source,
               output_format = output_format)
'''

INFO ilastik.app: Using tiktorch executable: ['/Applications/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/bin/python', '-m', 'tiktorch.server']
INFO ilastik.app: config file location: <none>
INFO ilastik.app: Starting ilastik from "/Applications/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/lib/python3.7".
Starting ilastik from "/Applications/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/lib/python3.7".


WARNING 2024-04-10 15:29:38,144 opConservationTracking 56124 8670007872 Could not find any ILP solver
WARNING 2024-04-10 15:29:38,154 opStructuredTracking 56124 8670007872 Could not find any ILP solver
WARNING 2024-04-10 15:29:38,157 structuredTrackingWorkflow 56124 8670007872 Could not find any learning solver. Tracking will use flow-based solver (DPCT). Learning for tracking will be disabled!


INFO ilastik.shell.projectManager: Opening Project: /Users/titouanlegourrierec/Desktop/labelisation3.ilp
INFO ilastik.workflows.pixelClassification.pixelClassificationWorkflow: Beginning Batch Processing
INFO ilastik.applets.dataSelection.dataSelectionApplet: Using axistags from previous lane: [y x c, None]
INFO ilastik.applets.batchProcessing.batchProcessingApplet: Exporting to /Users/titouanlegourrierec/Desktop/ilastik_result/664_leaf3_Simple_Segmentation.png
INFO lazyflow.utility.bigRequestStreamer: Estimated RAM usage per pixel is 212.0B * safety factor (2.0)
INFO lazyflow.utility.bigRequestStreamer: determining blockshape assuming available_ram is 10.9GiB, split between 8 threads
INFO lazyflow.utility.bigRequestStreamer: Chose blockshape: (4169, 831, 4)
INFO lazyflow.utility.bigRequestStreamer: Estimated RAM usage per block is 1.4GiB
DEBUG lazyflow.operators.classifierOperators: Features took 1.6230820000000001 seconds. Prediction took 10.302567 seconds. Subregion: start '[0, 0, 0

'\ndisplay_images(image_path = image_path,\n               result_base_path = result_base_path,\n               export_source = export_source,\n               output_format = output_format)\n'

In [7]:
image_path = "/Users/titouanlegourrierec/Desktop/Sheet47_leaf2.jpg"
ilastik_script_path = "/Applications/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/run_ilastik.sh"
project_path = "/Users/titouanlegourrierec/Desktop/selection_feuilles/Sheet77_leaf3.jpg"
result_base_path = "/Users/titouanlegourrierec/Desktop/ilastik_result/"
export_source = "Simple Segmentation"
output_format = "png"

start_time = time.time()
run_ilastik(image_path = image_path,
            ilastik_script_path = ilastik_script_path,
            project_path = project_path,
            result_base_path = result_base_path,
            export_source = export_source,
            output_format = output_format)
end_time = time.time()
print(f"Execution time: {(end_time - start_time):.3f} seconds")


display_images(image_path = image_path,
               result_base_path = result_base_path,
               export_source = export_source,
               output_format = output_format)

INFO ilastik.app: Using tiktorch executable: ['/Applications/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/bin/python', '-m', 'tiktorch.server']
INFO ilastik.app: config file location: <none>
INFO ilastik.app: Starting ilastik from "/Applications/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/lib/python3.7".
Starting ilastik from "/Applications/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/lib/python3.7".


WARNING 2024-04-09 18:56:55,632 opConservationTracking 61840 8602210880 Could not find any ILP solver
WARNING 2024-04-09 18:56:55,640 opStructuredTracking 61840 8602210880 Could not find any ILP solver
WARNING 2024-04-09 18:56:55,643 structuredTrackingWorkflow 61840 8602210880 Could not find any learning solver. Tracking will use flow-based solver (DPCT). Learning for tracking will be disabled!


INFO ilastik.shell.projectManager: Opening Project: /Users/titouanlegourrierec/Desktop/selection_feuilles/Sheet77_leaf3.jpg


ERROR 2024-04-09 18:56:56,666 excepthooks 61840 8602210880 Unhandled exception in thread: 'MainThread'
ERROR 2024-04-09 18:56:56,667 excepthooks 61840 8602210880 Traceback (most recent call last):
  File "/Applications/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/lib/python3.7/site-packages/ilastik/shell/projectManager.py", line 150, in openProjectFile
    hdf5File = h5py.File(projectFilePath, mode)
  File "/Applications/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/lib/python3.7/site-packages/h5py/_hl/files.py", line 533, in __init__
    fid = make_fid(name, mode, userblock_size, fapl, fcpl, swmr=swmr)
  File "/Applications/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/lib/python3.7/site-packages/h5py/_hl/files.py", line 226, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 106, in h5py.h

Error during conversion :  Command '['/Applications/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/run_ilastik.sh', '--headless', '--project=/Users/titouanlegourrierec/Desktop/selection_feuilles/Sheet77_leaf3.jpg', '--export_source=Simple Segmentation', '--output_format=png', '--output_filename_format=/Users/titouanlegourrierec/Desktop/ilastik_result/{nickname}_Simple_Segmentation', '/Users/titouanlegourrierec/Desktop/Sheet47_leaf2.jpg']' returned non-zero exit status 1.


RuntimeError: Error during Ilastik execution. See console output for details.

In [9]:
image_path = "/Users/titouanlegourrierec/Desktop/selection_feuilles/TITOUAN/111_leaf2.jpg"
ilastik_script_path = "/Applications/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/run_ilastik.sh"
project_path = "/Users/titouanlegourrierec/Desktop/labelisation3.ilp"
result_base_path = "/Users/titouanlegourrierec/Desktop/ilastik_result/"
export_source = "Simple Segmentation"
output_format = "png"

# Arguments to execute Ilastik in headless mode
ilastik_args = [
    ilastik_script_path,
    "--headless",
    "--project=" + project_path,
    "--export_source=" + export_source,
    "--output_format=" + output_format,
    "--output_filename_format=" + result_base_path + "{nickname}_" + export_source.replace(" ", "_"),
    *image_paths #*image_paths
]

# Execute the Ilastik command in headless mode with the specified arguments
try:
    subprocess.run(ilastik_args, check=True)
    print("Conversion completed successfully")
except subprocess.CalledProcessError as e:
    print("Error during conversion : ", e)
    raise RuntimeError("Error during Ilastik execution. See console output for details.")

INFO ilastik.app: Using tiktorch executable: ['/Applications/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/bin/python', '-m', 'tiktorch.server']
INFO ilastik.app: config file location: <none>
INFO ilastik.app: Starting ilastik from "/Applications/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/lib/python3.7".
Starting ilastik from "/Applications/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/lib/python3.7".


WARNING 2024-04-09 22:14:51,945 opConservationTracking 4188 8601047616 Could not find any ILP solver
WARNING 2024-04-09 22:14:51,956 opStructuredTracking 4188 8601047616 Could not find any ILP solver
WARNING 2024-04-09 22:14:51,959 structuredTrackingWorkflow 4188 8601047616 Could not find any learning solver. Tracking will use flow-based solver (DPCT). Learning for tracking will be disabled!


INFO ilastik.shell.projectManager: Opening Project: /Users/titouanlegourrierec/Desktop/labelisation3.ilp
INFO ilastik.workflows.pixelClassification.pixelClassificationWorkflow: Beginning Batch Processing
INFO ilastik.applets.dataSelection.dataSelectionApplet: Using axistags from previous lane: [y x c, None]
INFO ilastik.applets.batchProcessing.batchProcessingApplet: Exporting to /Users/titouanlegourrierec/Desktop/ilastik_result/664_leaf3_Simple_Segmentation.png
INFO lazyflow.utility.bigRequestStreamer: Estimated RAM usage per pixel is 212.0B * safety factor (2.0)
INFO lazyflow.utility.bigRequestStreamer: determining blockshape assuming available_ram is 11.0GiB, split between 8 threads
INFO lazyflow.utility.bigRequestStreamer: Chose blockshape: (4174, 831, 4)
INFO lazyflow.utility.bigRequestStreamer: Estimated RAM usage per block is 1.4GiB
DEBUG lazyflow.operators.classifierOperators: Features took 0.487791 seconds. Prediction took 5.998545 seconds. Subregion: start '[8348, 0, 0]' stop 